In [7]:
import cv2
import numpy as np
import imageio as io
import glob

#Load Yolo
def yolo(img_s, n):
    net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
    classes = []
    with open("yolov3.txt") as f:
        classes = [line.strip() for line in f.readlines()]

    layer_names = net.getLayerNames()      
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    colors = np.random.uniform(0, 255, size=(len(classes), 3))


    #Load image
    img = cv2.imread(img_s)
    height, width, channels = img.shape
    size = (width,height)

    #Detecting objects
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop = False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)


    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            color = colors[i]
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, label, (x, y + 30), font, 3, color, 3)
            
#     name = str(n) + '.jpg'
#     io.imsave('results/' + name, img)
    
def create_vid():
    width = 0
    height = 0
    size = ()
    vid_arr = []
    for file in glob.glob("results/*.jpg"):
            img_v = cv2.imread(file)
            vid_arr.append(img_v)
            height, width, layers = img_v.shape
            size = (width,height)
    out = cv2.VideoWriter('project.avi',cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
    for i in range(len(vid_arr)):
        out.write(vid_arr[i])
    out.release()

    
def main():
    files = []
    for file in glob.glob("images/*.jpg"):
        files.append(file)
#     for file in range(len(files)):
#         yolo(files[file], file)
    create_vid()
    print("Done")
    
main()


Done
